<a href="https://colab.research.google.com/github/mohammadsanaee/ML_in_application/blob/main/13_models_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf harbour-space-text-mining-course
!git clone https://github.com/horoshenkih/harbour-space-text-mining-course.git
import sys
sys.path.append('harbour-space-text-mining-course')
!pip install -q catboost
!pip install -q shap

from tmcourse.utils import (
    display_token_importance,
)

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import HTML, display
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(15,10)})

Cloning into 'harbour-space-text-mining-course'...
remote: Enumerating objects: 738, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (105/105), done.


# Linear models: feature effect

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]
remove = ("headers", "footers", "quotes")

data_train = fetch_20newsgroups(
    subset="train",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=remove,
)

data_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=remove,
)

# order of labels in `target_names` can be different from `categories`
target_names = data_train.target_names

# split target in a training set and a test set
y_train, y_test = data_train.target, data_test.target

# Extracting features from the training data using a sparse vectorizer

vectorizer = TfidfVectorizer(
    sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
)
X_train = vectorizer.fit_transform(data_train.data)


# Extracting features from the test data using the same vectorizer
X_test = vectorizer.transform(data_test.data)

feature_names = vectorizer.get_feature_names_out()

In [ ]:
from sklearn.linear_model import RidgeClassifier

clf = RidgeClassifier(tol=1e-2, solver="sparse_cg")
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
# learned coefficients weighted by frequency of appearance
average_feature_effects = clf.coef_ * np.asarray(X_train.mean(axis=0)).ravel()

for i, label in enumerate(target_names):
    top5 = np.argsort(average_feature_effects[i])[-5:][::-1]
    if i == 0:
        top = pd.DataFrame(feature_names[top5], columns=[label])
        top_indices = top5
    else:
        top[label] = feature_names[top5]
        top_indices = np.concatenate((top_indices, top5), axis=None)
top_indices = np.unique(top_indices)
predictive_words = feature_names[top_indices]

# plot feature effects
bar_size = 0.25
padding = 0.75
y_locs = np.arange(len(top_indices)) * (4 * bar_size + padding)

fig, ax = plt.subplots(figsize=(10, 8))
for i, label in enumerate(target_names):
    ax.barh(
        y_locs + (i - 2) * bar_size,
        average_feature_effects[i, top_indices],
        height=bar_size,
        label=label,
    )
ax.set(
    yticks=y_locs,
    yticklabels=predictive_words,
    ylim=[
        0 - 4 * bar_size,
        len(top_indices) * (4 * bar_size + padding) - 4 * bar_size,
    ],
)
ax.legend(loc="lower right")

print("top 5 keywords per class:")
print(top)

# Feature importance for Random Forest

We generate a synthetic dataset with only 3 informative features.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=3,
    n_redundant=0,
    n_repeated=0,
    n_classes=2,
    random_state=0,
    shuffle=False,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

feature_names = [f"feature {i}" for i in range(X.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)

Feature importances are provided by the fitted attribute feature_importances_ and they are computed as the mean and standard deviation of accumulation of the impurity decrease within each tree.

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

Let’s plot the impurity-based importance.

In [ ]:
forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

# Permutation importance

Permutation feature importance overcomes limitations of the impurity-based feature importance: they do not have a bias toward high-cardinality features and can be computed on a left-out test set.

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    forest, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

forest_importances = pd.Series(result.importances_mean, index=feature_names)

In [ ]:
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

As we see, permutation importance gives lower weights to random high-cardinality features.

# CatBoost: feature importance

In [ ]:
from catboost import CatBoostClassifier, Pool

clf = CatBoostClassifier()
clf.fit(X_train, y_train, verbose=False)

In [ ]:
feature_importances = clf.get_feature_importance(
    Pool(X_test, y_test, feature_names=feature_names),
    prettified=True,
)
feature_importances

In [ ]:
def visualize_feature_importances(clf, pool):
    feature_importances = clf.get_feature_importance(pool, prettified=True)

    sns.barplot(x="Importances", y="Feature Id", data=feature_importances)

visualize_feature_importances(
    clf,
    Pool(X_test, y_test, feature_names=feature_names)
)

# CatBoost: feature interactions

In [ ]:
def visualize_feature_interactions(clf, pool, feature_names):
    feature_interactions = clf.get_feature_importance(pool, type="Interaction")

    interactions = []
    feature_pairs = []
    for fi in feature_interactions:
        interactions.append(fi[2])
        feature_pairs.append(feature_names[int(fi[0])] + '__' + feature_names[int(fi[1])])
    feature_interactions_df = pd.DataFrame({
        'interaction': interactions,
        'feature_pair': feature_pairs,
    })

    sns.barplot(x="interaction", y="feature_pair", data=feature_interactions_df)

visualize_feature_interactions(
    clf,
    Pool(X_test, y_test, feature_names=feature_names),
    feature_names
)

# SHAP values

Get the data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/horoshenkih/harbour-space-ds210/master/datasets/california_housing_prices.csv')
df.head()

In [ ]:
X = df.drop('median_house_value', axis=1)
y = df.median_house_value
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from catboost import CatBoostRegressor
clf = CatBoostRegressor()
clf.fit(X_train, y_train, cat_features=['ocean_proximity'], verbose=False, plot=False)

SHAP algorithm is implemented in `shap` package.

It works witn CatBoost and supports categorical features.

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(clf)
pool_test = Pool(X_test, y_test, cat_features=['ocean_proximity'])
shap_values = explainer.shap_values(pool_test)

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X_test)

Look at an individual sample

In [ ]:
shap.initjs()

sample_idx = 1000
shap.force_plot(explainer.expected_value, shap_values[sample_idx,:], X_test.iloc[sample_idx,:])

# Masking and occlusion

In [ ]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(
    subset="all",
    remove=('headers', 'footers', 'quotes'),
    categories=("sci.space", "rec.autos", "talk.politics.misc", "comp.graphics")
)

texts = dataset.data
y = dataset.target

In [ ]:
texts_train, texts_test, y_train, y_test = train_test_split(texts, y, test_size=0.5, random_state=1)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("vec", TfidfVectorizer(stop_words="english")),
    ("clf", SGDClassifier(alpha=.0001, max_iter=50, loss="log", random_state=0)),
])

clf.fit(texts_train, y_train)

In [ ]:
def compute_token_importance(clf, text):
    from scipy.linalg import norm
    # a simple tokenizer: split by whitespace
    # not necessarily the same tokenizer that the vectorizer uses!
    import re
    tokens = re.split(r'\W+', text.strip())
    initial_text = " ".join(tokens)
    # predict the probabilities of all classes for the initial text
    initial_class_distribution = clf.predict_proba([initial_text])[0]
    token_importances = []
    for i, token in enumerate(tokens):
        masked_tokens = tokens[:]
        # mask (remove) one token
        masked_tokens[i] = "<UNKNOWN>"
        masked_text = " ".join(masked_tokens)
        # classify the text with masked token
        masked_class_distribution = clf.predict_proba([masked_text])[0]
        # token importance is the distance between
        # the initial distribution of classes
        # and the distribution after removal of the token
        token_importance = norm(initial_class_distribution - masked_class_distribution)
        token_importances.append((token, token_importance))
    return token_importances

# remember the initial categories in the dataset
print("categories:", ", ".join(dataset.target_names))
for text, label in zip(texts_test[:20], y_test[:20]):
    display(HTML("<hr>"))
    display(HTML(dataset.target_names[label]))  # uncomment this line to see the true label
    token_importances = compute_token_importance(clf, text)
    display_token_importance(token_importances)
    display(HTML("<hr>"))

# Visualizing layer activations in ConvNets

Get images of some cute animals.

In [ ]:
import torchvision

In [ ]:
data = torchvision.datasets.OxfordIIITPet(
    root="./data",
    download=True,
    transform=torchvision.transforms.ToTensor(),
)

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import numpy as np

image_index = 31
img = data[image_index][0]
plt.imshow(np.asarray(F.to_pil_image(img)))

Load pre-trained ResNet model and check that it works as expected

In [ ]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

PyTorch provides tools for model inspection: `get_graph_node_names` and `create_feature_extractor`

In [ ]:
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
train_nodes, eval_nodes = get_graph_node_names(model)
eval_nodes[:10]

Extract the output of the first ReLU. Check that there are sparse activations.

In [ ]:
extract_relu = create_feature_extractor(model, {'relu': 'layer1'})
img2 = extract_relu(batch)['layer1']

fig, ax = plt.subplots(8, 8, figsize=(15,15))
ax = ax.ravel()
for i in range(img2.shape[1]):
    ax[i].imshow(img2[0][i].detach().numpy())
plt.show()

Compare with uninitialized model - without training, all activations are dense

In [ ]:
extract_relu = create_feature_extractor(resnet50(), {'relu': 'layer1'})
img2 = extract_relu(batch)['layer1']

fig, ax = plt.subplots(8, 8, figsize=(15,15))
ax = ax.ravel()
for i in range(img2.shape[1]):
    ax[i].imshow(img2[0][i].detach().numpy())
plt.show()

# Home Assignment 10

In this assignment, you will work with probably the most famous and well-studied dataset.

Your task is to predict the value of `Survived` column (binary classification).

You can read more about the data here: https://www.kaggle.com/c/titanic/data

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
df.head()

## Exercise 1 (2 points)

Create training dataset, train CatBoost model and **find feature importances** for all features. You need to decide which columns to use as features. Feel free to do feature engineering.

In [ ]:
# YOUR CODE HERE

## Exercise 2 (2 points)

Compute feature interactions.



In [ ]:
# YOUR CODE HERE

## Exercise 3 (2 points)

Compute SHAP values. Use `shap.summary_plot`. Interpret the results.

In [ ]:
# YOUR CODE HERE